# Assignment: Credit Card Fraud Detection revised

we return to the credit card use case (last time we used Random Forests).


## Pre-Stage
Use your pre-processing from the last assignment as input.


In [ ]:
#download data
!unzip creditcard.csv.zip

In [2]:
# your preprocessing code

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('creditcard.csv')
pd.set_option('display.float', '{:.3f}'.format)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000,-1.360,-0.073,2.536,1.378,-0.338,0.462,0.240,0.099,0.364,...,-0.018,0.278,-0.110,0.067,0.129,-0.189,0.134,-0.021,149.620,0
1,0.000,1.192,0.266,0.166,0.448,0.060,-0.082,-0.079,0.085,-0.255,...,-0.226,-0.639,0.101,-0.340,0.167,0.126,-0.009,0.015,2.690,0
2,1.000,-1.358,-1.340,1.773,0.380,-0.503,1.800,0.791,0.248,-1.515,...,0.248,0.772,0.909,-0.689,-0.328,-0.139,-0.055,-0.060,378.660,0
3,1.000,-0.966,-0.185,1.793,-0.863,-0.010,1.247,0.238,0.377,-1.387,...,-0.108,0.005,-0.190,-1.176,0.647,-0.222,0.063,0.061,123.500,0
4,2.000,-1.158,0.878,1.549,0.403,-0.407,0.096,0.593,-0.271,0.818,...,-0.009,0.798,-0.137,0.141,-0.206,0.502,0.219,0.215,69.990,0


In [5]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)

In [6]:
from imblearn.over_sampling import SMOTE

X_samp, y_samp = SMOTE(random_state=1).fit_sample(X_train,y_train)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_samp= scaler.fit_transform(X_samp)

# Task: Train a MLP-Network
* Train and evaluate the model 
* Compare the results to the Random Forrest 
* Tune the hyper-parameters (like number of layers, neurons per layer, learning-rate and number of epochs ) to optimize the results

See: https://keras.io/api/

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential([
    Dense(units=64, kernel_initializer='normal', input_dim=30, activation='relu'),
    Dense(units=64, kernel_initializer='normal', activation='relu'),
    #Dropout(0.25),
    Dense(64, kernel_initializer='normal', activation='relu'),
    Dense(64, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1984      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 14,529
Trainable params: 14,529
Non-trainable params: 0
_________________________________________________________________


In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m, precision_m, recall_m])

In [11]:
model.fit(X_samp, y_samp, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
6397/6397 [==============================] - 9s 1ms/step - loss: 0.0251 - accuracy: 0.9919 - f1_m: 0.9902 - precision_m: 0.9932 - recall_m: 0.9884 - val_loss: 0.0055 - val_accuracy: 0.9984 - val_f1_m: 0.9992 - val_precision_m: 1.0000 - val_recall_m: 0.9984
Epoch 2/10
6397/6397 [==============================] - 8s 1ms/step - loss: 0.0071 - accuracy: 0.9979 - f1_m: 0.9976 - precision_m: 0.9970 - recall_m: 0.9982 - val_loss: 0.0040 - val_accuracy: 0.9989 - val_f1_m: 0.9994 - val_precision_m: 1.0000 - val_recall_m: 0.9989
Epoch 3/10
6397/6397 [==============================] - 9s 1ms/step - loss: 0.0052 - accuracy: 0.9986 - f1_m: 0.9984 - precision_m: 0.9979 - recall_m: 0.9990 - val_loss: 0.0011 - val_accuracy: 0.9996 - val_f1_m: 0.9998 - val_precision_m: 1.0000 - val_recall_m: 0.9996
Epoch 4/10
6397/6397 [==============================] - 8s 1ms/step - loss: 0.0040 - accuracy: 0.9989 - f1_m: 0.9987 - precision_m: 0.9984 - recall_m: 0.9991 - val_loss: 0.0030 - val_accuracy: 0.9

In [12]:
# evaluate the model
score= model.evaluate(X_test, y_test, verbose=0)  #loss_score, accuracy_score, f1_score, precision_score, recall_score = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', score[1])


Test accuracy: 0.9981390833854675
